Title: Discrete Soft Actor Critic with Prioritized Experience Replay Buffer

# Уведомление об авторском праве
Автор оригинальной работы: Совик Сергей, 2020 Июнь <sergeisovik@yahoo.com>

Предоставляется разрешение на использование, копирование, изменение и/или распространение данного программного обеспечения для любых целей с оплатой или без оплаты при условии, что вышеупомянутое уведомление об авторских правах и это уведомление о разрешении присутствуют во всех копиях.

Программное обеспечение предоставляется `как есть`, и автор отказывается от всех гарантий в отношении этого программного обеспечения, включая все подразумеваемые гарантии коммерческой выгоды и пригодности. Ни при каких обстоятельствах автор не несет ответственности за какие-либо особые, прямые или косвенные убытки или любые убытки, возникшие в результате потери данных или прибыли при использовании, будь то в результате действия договора, халатности или других вредоносных действий, возникающих в результате или в связи с использованием или работой данного программного обеспечения.

# Предисловие
Алгоритм пришлось реализовать на старых функциях Tensorflow с отключеным Eager режимом, в силу того, что новые функции Tensorflow 2.2 приводили к большой утечке памяти и выжирали 32 гб моей памяти буквально за один час.

Все названия переменным и классам даны исходя из опыта программирования больше 25 лет, возможно непривычном, но интуитивно понятном для читателей виде и не являются общепринятым стандартом.

Данная статья написана с целью расширения круга пользователей алгоритмом `Soft Actor Critic`, т.к. на момент написания статьи он является самым лучшим, а все существующие статьи написаны непонятным для многих программистов математическим языком.

Для тех, кто совершенно не понимает, что такое `Вычислительный граф` и не хочет вдаваться в детальные подробности. Это модель описывающая взаимосвязь между всеми вычислениями, в том числе определяющая их порядок выполнения. Каждая операция в котором называется `Узел`. `Вычислительный граф` чемто напоминает блок схему с множеством возможных входов и выходов. Таким образом, запрашивая вычислить результат `Узла` у нейросетевого движка, производится вычисление всех зависимостей, и при необходимости запрашиваются входные данные.

# Алгоритм `Genetic Soft Actor Critic`
Алгоритм реализован в виде одного цельного графа, что позволяет уменьшить объём обмена данных с GPU, и ускорить процесс обучения.

Алгоритм состоит из четырёх основных блоков:
- Блок `Нейросеть`
- Блок `Игрок`
- Блок `Генетический буфер повтора`
- Блок `Тренер`

Каждый из которых может работать параллельно-последовательно.

## Блок `Нейросеть`
Нейросеть состоит из нескольких независимо обучаемых блоков:
- Две подсети копии `Актёр-тренер` и `Актёр-цель`
- Две дублирующие подсети `Критик-тренер`
- Две подсети `Критик-цель`
- Коэффициент `Альфа-регулятор`.

### Две подсети копии `Актёр-тренер` и `Актёр-цель`
`Актёр-цель` используется исключительно для возможности распараллеливать обучение и заполнение `Буфера повтора` новыми данными и является полной копией нейросети `Актёр-тренер`.

### Две дублирующие подсети `Критик-тренер`
Необходимы для минимизации ошибки.

### Две подсети `Критик-цель`
Используются для плавного обучения методом `скользящее среднее`.

### Коэффициент `Альфа-регулятор`
Исполняет роль микроподстройки процесса обучения, для увеличения точности.

## Блок `Игрок`
Существует некая среда, в которой необходимо производить определённые действия для достижения поставленной цели. Для упрощения понимания, назовём среду `Игрой`. Задача `Игрока` собирать данные наблюдения от `Игры`, совершать действия, и получать от `Игры` `Награду` или производить самостоятельно `Оценку` этих действий. Каждое совершённое действие, это ход. За один ход мы имеем следующий набор данных: `Прошлое наблюдение`, `Текущее наблюдение`, `Совершённое действие`, `Награда` или `Оценка`, `Cтатус конца`. Решение, о том, какое совершать действие принимает сеть `Актёр-цель` на основе данных `Прошлого наблюдения`. Если принятое решение приводит к ситуации, которую можно считать концом, то `Игрок` завершает и сбрасывает `Игру`.

Оценка действий бывает двух типов:
- Оценка награды за каждый шаг
- Оценка всего эпизода

Каждый ход записывается в `Буфер повтора` для дальнейшего обучения и называется `Траекторией`.

### Оценка награды за каждый шаг
`Игрок` совершает действие и сразу производит запись в `Буфер повтора` следующие показатели: `Прошлое наблюдение`, `Текущее наблюдение`, `Cовершённое действие`, `Награда`. Тогда `Оценку` производит `Тренер`.

### Оценка всего эпизода
`Игрок` совершает действия и производит запись во `Временный буфер`. По окончании эпизода рассчитывает `Оценку` на каждом ходу и после производит запись в `Буфер повтора` всего эпизода со следующими показателями: `Прошлое наблюдение`, `Текущее наблюдение`, `Cовершённое действие`, `Оценка`.

## Блок `Генетический буфер повтора`
Представляет из себя циклический `Буфер повтора`, который при переполнении начинает перезатирать более старые данные более новыми. А также включает в себя `Древовидный буфер суммы` и `Древовидный буфер максимума`, используемые для расчёта приоритета каждого хода хранящегося в `Буфере повтора`. Древовидные буферы в паре помогают реализовать подобие генетического алгоритма при выборе данных из `Буфера повтора`, что позволяет сильно ускорить процесс обучения, а также уменьшает вероятность сбивания или зависания обучаемой модели в плохом состоянии. Плохое состояние может быть следствием привыкания нейросети к плохим результатам.

## Блок `Тренер`
Основной мозговой центр алгоритма, который управляет всеми остальными блоками.

Цикл обучения на каждый шаг `Игрока`:
1. Выбрать блок `Траекторий` из `Генетический буфера повтора` с учётом приоритетов.
2. Обучить независимо два `Критик-тренера`.
3. Обновить `Критик-цели` методом `скользящее среднее`.
4. Обучить `Актёр-тренер` и `Альфа-регулятор`.
5. Обновить `Актёр-цель`.
6. Обновить приоритеты в `Генетическом буфере повтора` для обработанных ходов из выбранного блока.

Процесс обучения стандартный: прямое распространение, вычисление потери, вычисление градиентов, обратное распространение.

# Пример результатов тренировки `LunarLander-v2`
<table style="float:left;">
    <tr>
        <td style="text-align: center;">Средний счёт за эпизод</td>
        <td style="text-align: center;">Среднее число шагов за эпизод</td>
    </tr><tr>
        <td><img src="GSAC-Score.svg" width="320pt"></td>
        <td><img src="GSAC-Steps.svg" width="320pt"></td>
    </tr>
</table>

# Параметры

In [1]:
# Использовать GPU?
bGPU = False
# Размер одного слоя нейросети `Кодер`
uEncoderLayerSize = 64
# Имя модели и логов
sName = "%d" % uEncoderLayerSize
# Максимальное количество шагов в эпизоде
uEpisodeStepLimit = 1024
# Размер буфера повтора, должен вмещать хотябы 100 эпизодов
uReplayCapacity = 128 * 1024
# Размер пакета данных при обучении
uBatchSize = 256
# Восстановить граф из сохранения?
bRestore = False
# Производить вывод на экран?
bRender = False
# Производить оценку всего эпизода?
bEpisodeRating = False
# Делать выборку из буфера повтора с учётом приоритетов?
bPriorityMode = False
# Фактор забывания оценки
fRatingDiscountFactor = 0.99
# Коэфициент обновления целевой нейронной сети
fTrainableTargetAverageUpdateCoef = 0.005
# Уровень логирования и ведения статистики
uLogLevel = 2

# Подготовка рабочего пространства
Подключение модулей и их настройка.

In [2]:
# Отключаем спам в консоль от Tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
# Отключаем работу с GPU для маленьких сетей, т.к. CPU работает быстрее
if not bGPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Математика
import numpy as np
import math

# Tensorflow
import tensorflow as tf
import tensorflow._api.v2.compat.v1 as tf1
import tensorflow.keras.layers as kl
import tensorflow.keras.optimizers as ko
import tensorflow.keras.initializers as ki

# Отключить выделение всей доступной памяти
if bGPU:
    aoGPUPhysicalList = tf.config.experimental.list_physical_devices('GPU')
    if aoGPUPhysicalList:
        try:
            for oGPUDevice in aoGPUPhysicalList:
                tf.config.experimental.set_memory_growth(oGPUDevice, True)
        except RuntimeError as e:
            pass

# Указываем тип данных по умолчанию для сетей keras
tf.keras.backend.set_floatx('float32')

# Ручной контроль создания графов и управления сессиями, чтобы убирать проблему утечки памяти
tf1.disable_eager_execution()

# Дата и время
import time
from datetime import datetime

# Виртуальное игровое окружение для `Обучения с подкреплением`
import gym

# Поддержка абтрактных методов
import abc

In [3]:
import os
import psutil

# Получить информацию об использовании памяти текущим процессом
def getMemoryUsage():
    oProcess = psutil.Process(os.getpid())
    return "%d:%d" % (oProcess.memory_info().rss, oProcess.memory_info().vms)

# Надстройка Tensorflow
Набор функций и классов упрощающих ручную работу с графами и сессиями _Tensorflow_

In [4]:
# Типы по умолчанию
tfFloat = tf.keras.backend.floatx()
tfInt = tf.int32

# Инициализатор массивов переменных в графе (для внутреннего использования)
class ArrayInitializer(ki.Initializer):
    def __init__(self, aValue):
        super(ArrayInitializer, self).__init__()
        self.aValue = aValue

    def __call__(self, shape, dtype=None):
        return tf.convert_to_tensor(self.aValue, dtype=dtype)

# Создать инициализатор в зависимости от формы и значения (для внутреннего использования)
def tfInitializer(tShape, oValue=None):
    if (oValue is not None) and (not isinstance(oValue, ki.Initializer)):
        if isinstance(oValue, list):
            oValue = np.array(oValue)
        if type(oValue) is np.ndarray:
            if oValue.shape != tShape:
                raise TypeError("Неверный размер инициализатора %s, вместо %s" % (oValue.shape, tShape))
            return ArrayInitializer(oValue)
        elif oValue == 0:
            return ki.Zeros()
        elif oValue == 1:
            return ki.Ones()
        else:
            return ki.Constant(oValue)
    return oValue

# Объявить локальную переменную, используемую внутри одного функционального блока графа
def tfLocalVariable(sName, oType, oValue=None, bTrainable=False):
    if isinstance(oType, tuple):
        sType, tShape = oType[0], oType[1]
    else:
        sType, tShape = oType, []
    oValue = tfInitializer(tShape, oValue)
    return tf1.get_local_variable(sName, shape=tShape, dtype=sType, initializer=oValue, trainable=bTrainable, use_resource=True)

# Объявить глобальную переменную, используемую всем графом в течении сессии
def tfGlobalVariable(sName, oType, oValue=None, bTrainable=False):
    if isinstance(oType, tuple):
        sType, tShape = oType[0], oType[1]
    else:
        sType, tShape = oType, []
    oValue = tfInitializer(tShape, oValue)
    return tf1.get_variable(sName, shape=tShape, dtype=sType, initializer=oValue, trainable=bTrainable, use_resource=True)

# Объявить константу
def tfConstant(sName, oType, oValue):
    sType = oType[0] if isinstance(oType, tuple) else oType
    return tf1.constant(oValue, dtype=sType, name=sName)

# Объявить узел входных данных графа
def tfInput(sName, oType):
    if isinstance(oType, tuple):
        sType, tShape = oType[0], oType[1]
    else:
        sType, tShape = oType, []
    return tf1.placeholder(sType, shape=tShape, name=sName)

# Псевдоним сокращение для функции ожидания вычисления набора операций
class tfWait(object):
    def __init__(self, aDependencies):
        self.tfControl = tf.control_dependencies(aDependencies)

    def __enter__(self):
        return self.tfControl.__enter__()

    def __exit__(self, clsErrorType, errValue, oTraceback):
        return self.tfControl.__exit__(clsErrorType, errValue, oTraceback)

# Текущая активная сессия
tfActiveSession = None

# Создать сессию для управления выполнением графа
class tfSession(object):
    def __init__(self, tfoGraph=None, sTarget=''):
        self.tfoSession = tf1.Session(target=sTarget, graph=tfoGraph.tfoGraph)

    def __enter__(self):
        global tfActiveSession
        self.tfoDefault = self.tfoSession.as_default()
        self.tfoDefault.__enter__()
        self.tfRestoreSession = tfActiveSession
        tfActiveSession = self
        return self

    def __exit__(self, clsErrorType, errValue, oTraceback):
        global tfActiveSession
        self.tfoSession.close()
        self.tfoDefault = None
        tfActiveSession = self.tfRestoreSession

    def initGlobal(self):
        self.tfoSession.run(tf1.global_variables_initializer())

    def initLocal(self):
        self.tfoSession.run(tf1.local_variables_initializer())

    def eval(self, tfoOutputTensor, oInputDictionary=None):
        return self.tfoSession.run(tfoOutputTensor, oInputDictionary)

# Вычислить результат в узле графа внутри текущей сессии
def tfEval(tfoOutputTensor, oInputDictionary=None):
    return tfActiveSession.eval(tfoOutputTensor, oInputDictionary)

# Произвести инициализацию глобальных переменных внутри текущей сессии
def tfInitGlobal():
    tfActiveSession.initGlobal()

# Произвести инициализацию локальных переменных внутри текущей сессии
def tfInitLocal():
    tfActiveSession.initLocal()

# Текущий активный граф
tfActiveGraph = None

# Создать граф
class tfGraph(object):
    def __init__(self):
        self.tfoGraph = tf.Graph()
        self.tfoDefault = None

    def __enter__(self):
        global tfActiveGraph
        self.tfoDefault = self.tfoGraph.as_default()
        self.tfoDefault.__enter__()
        self.tfRestoreGraph = tfActiveGraph
        tfActiveGraph = self
        return self

    def __exit__(self, clsErrorType, errValue, oTraceback):
        global tfActiveGraph
        self.tfoDefault = None
        tfActiveGraph = self.tfRestoreGraph
        
# Псевдоним сокращение для оптимизатора
class AMSgrad(ko.Adam):
    def __init__(self, lr=0.001):
        super(AMSgrad, self).__init__(lr=lr, amsgrad=True)

# Ограничение значений градиентов для предотвращения `inf` и `nan`
def fnClipGradients(aGradients, tfMaxNormal):
    aClippedGradients = []
    for tfoGrad in aGradients:
        if tfoGrad is not None:
            if isinstance(tfoGrad, tf.IndexedSlices):
                tfoTemp = tf.clip_by_norm(tfoGrad.values, tfMaxNormal)
                tfoGrad = tf.IndexedSlices(tfoTemp, tfoGrad.indices, tfoGrad.dense_shape)
            else:
                tfoGrad = tf.clip_by_norm(tfoGrad, tfMaxNormal)
        aClippedGradients.append(tfoGrad)
    return aClippedGradients

# Копирование весов из одной нейросети в другую
def fnHardUpdate(tafTargetVariables, tafSourceVariables):
    atopUpdates = []
    tfStrategy = tf.distribute.get_strategy()

    for (tfoTarget, tfoSource) in zip(tafTargetVariables, tafSourceVariables):
        def fnUpdate(tfoTarget, tfoSource):
            return tfoTarget.assign(tfoSource)

        if tf.distribute.has_strategy() and tfoTarget.trainable:
            topUpdate = tfStrategy.extended.update(tfoTarget, fnUpdate, args=(tfoSource,))
        else:
            topUpdate = fnUpdate(tfoTarget, tfoSource)

        atopUpdates.append(topUpdate)
    return tf.group(*atopUpdates)

# Наложение весов из одной нейросети в другую методом `скользящее среднее`
def fnSoftUpdate(tafTargetVariables, tafSourceVariables, tfZero, tfOne, tfTrainableTargetAverageForgetCoef, tfTrainableTargetAverageUpdateCoef):
    atopUpdates = []
    tfStrategy = tf.distribute.get_strategy()

    for (tfoTarget, tfoSource) in zip(tafTargetVariables, tafSourceVariables):
        def fnUpdate(tfoTarget, tfoSource):
            if not tfoTarget.trainable:
                tfTargetAverageForgetCoef = tfZero
                tfTargetAverageUpdateCoef = tfOne
            else:
                tfTargetAverageForgetCoef = tfTrainableTargetAverageForgetCoef
                tfTargetAverageUpdateCoef = tfTrainableTargetAverageUpdateCoef

            return tfoTarget.assign(tfoTarget * tfTargetAverageForgetCoef + tfoSource * tfTargetAverageUpdateCoef)

        if tf.distribute.has_strategy() and tfoTarget.trainable:
            topUpdate = tfStrategy.extended.update(tfoTarget, fnUpdate, args=(tfoSource,))
        else:
            topUpdate = fnUpdate(tfoTarget, tfoSource)

        atopUpdates.append(topUpdate)
    return tf.group(*atopUpdates)

# Записать детальную статистику тензора, одной переменной графа
def fnTensorSummary(oSummaryWriter, sTag, tfoVariable, tuStep):
    with oSummaryWriter.as_default():
        with tf.name_scope(sTag):
            return tf.group(
                tf.summary.histogram('Histogram', tfoVariable, tuStep),
                tf.summary.scalar('Mean', tf.reduce_mean(tfoVariable, 'fMean'), tuStep),
                tf.summary.scalar('MeanAbs', tf.reduce_mean(tf.abs(tfoVariable), 'fMeanAbs'), tuStep),
                tf.summary.scalar('Max', tf.reduce_max(tfoVariable), tuStep),
                tf.summary.scalar('Min', tf.reduce_min(tfoVariable), tuStep)
            )

# Записать детальную статистику весовых коэфициентов нейросети
def fnWeightsSummary(oSummaryWriter, zipWeightfoGradientsAndVariables, tuStep):
    aOps = []
    with oSummaryWriter.as_default():
        for tfaGradientsGroup, tfaVariablesGroup in zipWeightfoGradientsAndVariables:
            sGroupName = tfaVariablesGroup.name.replace(':', '_')

            if isinstance(tfaVariablesGroup, tf.IndexedSlices):
                tfaValues = tfaVariablesGroup.values
            else:
                tfaValues = tfaVariablesGroup
            aOps.append(tf.summary.histogram('Weights/' + sGroupName, tfaValues, tuStep))
            aOps.append(tf.summary.scalar('WeightsNorm/' + sGroupName, tf.linalg.global_norm([tfaValues]), tuStep))

            if tfaGradientsGroup is not None:
                if isinstance(tfaGradientsGroup, tf.IndexedSlices):
                    tfaGradients = tfaGradientsGroup.values
                else:
                    tfaGradients = tfaGradientsGroup
                aOps.append(tf.summary.histogram('Gradients/' + sGroupName, tfaGradients, tuStep))
                aOps.append(tf.summary.scalar('GradientsNorm/' + sGroupName, tf.linalg.global_norm([tfaGradients]), tuStep))
    return tf.group(*aOps)

# Набор функций для выбора действия в дискретных моделях

In [5]:
def fnSelectBest(tafUnnormalizedLogProbabilities):
    return tf.argmax(tafUnnormalizedLogProbabilities, axis=-1, output_type=tfInt)

def fnSelectRandom(tafUnnormalizedLogProbabilities):
    return tf.squeeze(tf.random.categorical(tafUnnormalizedLogProbabilities, 1, dtype=tfInt), axis=-1)

def fnSelectNoisyBest(tafUnnormalizedLogProbabilities):
    with tf1.variable_scope('Const', reuse=tf1.AUTO_REUSE):
        tfTotalMinLogInput = tfConstant('fMinLogInput', tfFloat, 1e-8)
        tfTotalMaxLogInput = tfConstant('fMaxLogInput', tfFloat, 1-1e-8)

    with tf.name_scope('fnSelectNoisyBest'):
        # Массив случайных чисел для добавления шума к логарифмическим вероятностям
        tafRandomUniform = tf.random.uniform(tafUnnormalizedLogProbabilities.shape, minval=tfTotalMinLogInput, maxval=tfTotalMaxLogInput, dtype=tfFloat, seed=None) # pylint: disable=unexpected-keyword-arg
        # Распределение Гумбеля для случайных чисел
        tafGumbel = -tf.math.log(-tf.math.log(tafRandomUniform)) # pylint: disable=invalid-unary-operand-type
        # Ненормализованные предполагаемые логарифмические вероятности возможных действий с добавлением шума
        tafUnnormalizedNoisyLogProbabilities = tafUnnormalizedLogProbabilities + tafGumbel

    return tf.argmax(tafUnnormalizedNoisyLogProbabilities, axis=-1, output_type=tfInt)

def fnSelectNoisyRandom(tafUnnormalizedLogProbabilities):
    with tf1.variable_scope('Const', reuse=tf1.AUTO_REUSE):
        tfTotalMinLogInput = tfConstant('fMinLogInput', tfFloat, 1e-8)
        tfTotalMaxLogInput = tfConstant('fMaxLogInput', tfFloat, 1-1e-8)

    with tf.name_scope('fnSelectNoisyRandom'):
        # Массив случайных чисел для добавления шума к логарифмическим вероятностям
        tafRandomUniform = tf.random.uniform(tafUnnormalizedLogProbabilities.shape, minval=tfTotalMinLogInput, maxval=tfTotalMaxLogInput, dtype=tfFloat, seed=None) # pylint: disable=unexpected-keyword-arg
        # Распределение Гумбеля для случайных чисел
        tafGumbel = -tf.math.log(-tf.math.log(tafRandomUniform)) # pylint: disable=invalid-unary-operand-type
        # Ненормализованные предполагаемые логарифмические вероятности возможных действий с добавлением шума
        tafUnnormalizedNoisyLogProbabilities = tafUnnormalizedLogProbabilities + tafGumbel

    return tf.squeeze(tf.random.categorical(tafUnnormalizedNoisyLogProbabilities, 1, dtype=tfInt), axis=-1)

# Модель среды
Модель среды определяет текущее состояние окружения, и на основе команд управления, вычисляет следующее состояние окружения, награду за совершённое действие, а также информирует о конце, т.е. необходимости сброса среды в начальное состояние.

In [6]:
# Базовый класс среды
class EnvironmentImpl(object):
    def __init__(self, uObservationSize, uActionsSize, bDiscrete):
        # Размер массива данных о состоянии наблюдения
        self.uObservationSize = uObservationSize
        # Размер массива действий
        self.uActionsSize = uActionsSize
        # Дискретный или непрерывный тип управления средой
        self.bDiscrete = bDiscrete

    # Сброс среды. Возвращает текущее состояние наблюдения
    @abc.abstractmethod
    def reset(self):
        pass

    # Следующий шаг состояния наблюдения. Возвращает новое состояние наблюдения, награду и состояние завершения
    @abc.abstractmethod
    def step(self, oAction):
        pass

In [7]:
# Модель среды для игры LunarLander
class CustomEnvironment(EnvironmentImpl):
    def __init__(self, bRender=True):
        sEnvironment = 'LunarLander-v2'
        self.oEnvironment = gym.make(sEnvironment)
        
        # Ограничение максимальной длительности игры
        # Должно быть хотябы в 4 раза больше предполагаемой средней длительности, но не слишком большым
        self.oEnvironment._max_episode_steps = uEpisodeStepLimit * 10
        
        tObservationShape = self.oEnvironment.observation_space.shape
        tActionsShape = (self.oEnvironment.action_space.n,)

        super(CustomEnvironment, self).__init__(
            np.prod(list(tObservationShape)),
            np.prod(list(tActionsShape)),
            True
        )

        self.bRender = bRender

    def reset(self):
        return self.oEnvironment.reset()

    def step(self, uAction):
        aObservation, fReward, bDone, _ = self.oEnvironment.step(uAction)

        if self.bRender:
            self.oEnvironment.render()

        oInfo = {}
        return aObservation, fReward, bDone, oInfo

    def close(self):
        self.oEnvironment.close()

# Модели нейросетей

In [8]:
# Базовая модель нейросети
class ModelImpl(tf.keras.Model):
    def __init__(self, sName=None):
        super(ModelImpl, self).__init__(name=sName)

    # Переопределение метода для того, чтобы Tensorflow объявил переменную `self._build_input_shape`
    #  `self._build_input_shape` используетя для логирования структуры модели нейросети
    def build(self, tInputShape):
        super(ModelImpl, self).build(tInputShape)

# ВАЖНО! Не использовать `relu`, т.к. это приводит к проблеме `исчезающего градиента`, близкого или равного нулю

# Нейросеть `Критик`, производит оценку возможной выгоды окружения среды при совершении определённых действий
class CriticNetwork(ModelImpl):
    def __init__(self, sName=None):
        super(CriticNetwork, self).__init__(sName=sName)
        # Блок `Выпрямитель`
        self.fnFlattenObservations = kl.Flatten()
        self.fnFlattenActions = kl.Flatten()
        self.fnConcatInput = kl.Concatenate()
        # Блок `Кодер`
        self.fnEncoder = tf.keras.Sequential()
        self.fnEncoder.add(kl.Dense(uEncoderLayerSize, activation='elu'))
        self.fnEncoder.add(kl.Dense(uEncoderLayerSize, activation='elu'))
        self.fnEncoder.add(kl.Dense(32, activation='elu'))
        # Блок `Критик`
        self.fnCritic = kl.Dense(1, activation='linear')

    # Создать переменные весов для нейросети
    def build(self, tObservationShape, tActionsShape):
        super(CriticNetwork, self).build((tObservationShape[0], np.prod(list(tObservationShape[1:])) + np.prod(list(tActionsShape[1:]))))

    # Подготовить данные с помощью блока `Выпрямитель` для вычисления результата нейросети
    def prepare(self, tafObservations, tafActions):
        tafFlatObservations = self.fnFlattenObservations.call(tafObservations)
        tafFlatActions = self.fnFlattenActions.call(tafActions)
        tafStates = self.fnConcatInput([tafFlatObservations, tafFlatActions])
        return tafStates

    # Вычислить результат нейросети (предсказать рейтинг)
    def call(self, tafStates):
        tafEncoded = self.fnEncoder(tafStates)
        tafPredRating = self.fnCritic(tafEncoded)
        return tafPredRating

# Нейросеть `Дискретный актёр`, генерирует ненормализованные логарифмические вероятности действий
class DiscreteActorNetwork(tf.keras.Sequential):
    def __init__(self, uActionCount, sName=None):
        super(DiscreteActorNetwork, self).__init__(name=sName)
        # Блок `Кодер`
        self.fnEncoder = tf.keras.Sequential()
        self.fnEncoder.add(kl.Dense(uEncoderLayerSize, activation='elu'))
        self.fnEncoder.add(kl.Dense(uEncoderLayerSize, activation='elu'))
        self.fnEncoder.add(kl.Dense(32, activation='elu'))
        # Блок `Актёр`
        self.fnAction = kl.Dense(uActionCount, activation='linear')
        # Последовательность вычислений
        self.add(self.fnEncoder)
        self.add(self.fnAction)

# Нейросеть `Непрерывный актёр`, генерирует среднее и стандартное отклонение возможного действия
class ContinuousActorNetwork(ModelImpl):
    def __init__(self, uActionCount, sName=None):
        super(ContinuousActorNetwork, self).__init__(sName=sName)
        # Блок `Кодер`
        self.fnEncoder = tf.keras.Sequential()
        self.fnEncoder.add(kl.Dense(uEncoderLayerSize, activation='elu'))
        self.fnEncoder.add(kl.Dense(uEncoderLayerSize, activation='elu'))
        self.fnEncoder.add(kl.Dense(32, activation='elu'))
        # Блок `Актёр`
        # Начальные значения устанавливаются в районе 0 с небольшим отклонением, для избежания резких и непредвиденных действий
        self.fnMean = kl.Dense(uActionCount, activation='linear',
            kernel_initializer=ki.VarianceScaling(0.1),
            bias_initializer=ki.Zeros(),)
        self.fnStd = kl.Dense(uActionCount, activation='linear',
            kernel_initializer=ki.VarianceScaling(0.1),
            bias_initializer=ki.Constant(0.0),)

    # Создать переменные весов для нейросети
    def build(self, tObservationShape):
        super(ContinuousActorNetwork, self).build(tObservationShape)

    # Вычисление результата нейросети
    def call(self, tafState):
        tafEncoded = self.fnEncoder(tafState)
        tafPredLocation = self.fnMean(tafEncoded)
        tafPredScale = self.fnStd(tafEncoded)
        return tafPredLocation, tafPredScale

# Создать нейросеть `Актёр` в зависимости от типа среды
def CreateActor(oEnvironment, uBatchSize=256, sName='Actor'):
    if oEnvironment.bDiscrete:
        nnActor = DiscreteActorNetwork(oEnvironment.uActionsSize, sName)
    else:
        nnActor = ContinuousActorNetwork(oEnvironment.uActionsSize, sName)
    nnActor.build((uBatchSize, oEnvironment.uObservationSize))
    return nnActor

# Управление средой

In [9]:
# Класс управления `Игрок`, используя нейросеть `Актёр` совершает дейтвия в виртуальной среде
# fnPolicy - функция преобразующая логарифмические вероятности в номер действия
class CustomPlayer(object):
    def __init__(self, oEnvironment, nnActor, fnPolicy=fnSelectBest):
        self.oEnvironment = oEnvironment
        self.nnActor = nnActor

        self.reset()

        # Константы необходимые для работы с графом
        with tf1.variable_scope('Const', reuse=tf1.AUTO_REUSE):
            tuActionsSize = tfConstant('uActionsSize', tfInt, self.oEnvironment.uActionsSize)
            tuZero = tfConstant('uZero', tfInt, 0)

        # Формирование блока графа для вычисления совершаемого действия
        with tf.name_scope('CustomPlayer'):
            # Функциональный блок `fnAction`
            with tf.name_scope('fnAction'):
                with tf1.variable_scope('Input', reuse=tf1.AUTO_REUSE):
                    # Узел входных данных наблюдения за средой
                    self.tinafReplayObservation = tfInput('afObservation', (tfFloat, [self.oEnvironment.uObservationSize]))

                # Вычислить ненормализованные логарифмические вероятности следующего действия
                tafActions = nnActor.call(self.tinafReplayObservation[None, :], training=False)
                self.tafActions = tf.squeeze(tafActions)
                # Преобразовать вероятности в номер действия используя функцию политики
                self.tuAction = tf.squeeze(tf.clip_by_value(fnPolicy(tafActions), tuZero, tuActionsSize))

    # Вычислить следующее действие
    def action(self, afObservation):
        return tfEval([self.tafActions, self.tuAction], {self.tinafReplayObservation: afObservation})

    # Сброс среды и состояния окружения
    def reset(self):
        afObservation = self.oEnvironment.reset()
        self.afPrevObservation = None
        self.uAction = None
        self.afActions = None
        self.afObservation = np.array(afObservation, dtype=tfFloat).flatten()
        self.uStep = 0
        self.fScore = 0
        self.fReward = 0
        self.fAverageReward = 0
        self.bDone = False

    # Совершить следующее предполагаемое действие
    def next(self):
        if not self.bDone:
            self.afActions, self.uAction = self.action(self.afObservation)

            if self.uStep >= uEpisodeStepLimit:
                self.uAction = 0
            
            afObservation, self.fReward, bDone, _ = self.oEnvironment.step(self.uAction)

            self.afPrevObservation = self.afObservation
            self.afObservation = np.array(afObservation, dtype=tfFloat).flatten()
            self.uStep += 1
            self.fScore += self.fReward
            self.fAverageReward = self.fAverageReward * 0.999 + self.fReward * 0.001
            if bDone:
                self.bDone = True
        return self.bDone

# Агент `Soft Actor Critic`

In [10]:
class SacAgent(object):
    def __init__(self,
            oPlayer,
            uReplayCapacity=128*1024,
            fRatingDiscountFactor=0.99,
            uBatchSize=256,
            bEpisodeRating=False,
            bPriorityMode=True,
            fTrainableTargetAverageUpdateCoef=0.005,
            tfTrainStepCounter=None,
            uLogLevel=1,
            sLogsPath='logs/',
            sRestorePath='models/'):

        if uReplayCapacity < 2:
            raise TypeError('uReplayCapacity должно быть больше 1, а задано %d' % uReplayCapacity)

        # Входные параметры

        self.oPlayer = oPlayer
        self.uBatchSize = uBatchSize
        self.bEpisodeRating = bEpisodeRating
        self.bPriorityMode = bPriorityMode
        self.fRatingDiscountFactor = fRatingDiscountFactor

        # Коэфициент сглаживания статистики среднего значния градиентов
        fGradientNormalUpdateCoef = 0.01
        # Максимальное среднее значение градиентов
        fMaxGradientNormal = 200

        self.uLogLevel = uLogLevel

        # Оптимизаторы сетей

        self.koActorOptimizer = AMSgrad(3e-4)
        self.koCriticOptimizer = AMSgrad(3e-4)
        self.koAlphaOptimizer = AMSgrad(3e-4)

        # Самописец статистики

        dtCurrentTime = datetime.now().strftime("%Y%m%d-%H%M%S")
        sLogPath = sLogsPath + dtCurrentTime
        self.oSummaryWriter = tf.summary.create_file_writer(sLogPath)

        # Временное хранилище ходов для целого эпизода

        if self.bEpisodeRating:
            # Текущий размер данных во временном буфере
            self.uBufferSize = 0
            # Текущая вместительность временного буфера
            self.uBufferCapacity = 256

            # Хранилище временного буфера: наблюдения, действия, награды
            self.afPrevObservations = np.zeros((self.uBufferCapacity, self.oPlayer.oEnvironment.uObservationSize), dtype=tfFloat)
            self.afObservations = np.zeros((self.uBufferCapacity, self.oPlayer.oEnvironment.uObservationSize), dtype=tfFloat)
            if self.oPlayer.oEnvironment.bDiscrete:
                self.auActions = np.zeros((self.uBufferCapacity,), dtype=np.int32)
            else:
                self.afActions = np.zeros((self.uBufferCapacity, self.oPlayer.oEnvironment.uActionsSize), dtype=tfFloat)
            self.afRewards = np.zeros((self.uBufferCapacity,), dtype=tfFloat)
            self.afRatings = np.zeros((self.uBufferCapacity,), dtype=tfFloat)

        # Списки переменных для сохранения на диск

        aTrainVariables = []
        aTargetVariables = oPlayer.nnActor.trainable_variables
        aReplayBufferVariables = []

        # Глобальные константы графа

        with tf1.variable_scope('Const', reuse=tf1.AUTO_REUSE):
            tfZero = tfConstant('fZero', tfFloat, 0)
            tfHalf = tfConstant('fHalf', tfFloat, 0.5)
            tfOne = tfConstant('fOne', tfFloat, 1)
            tuOne = tfConstant('uOne', tfInt, 1)
            if self.bPriorityMode:
                tuTwo = tfConstant('uTwo', tfInt, 2)

            tuActionsSize = tfConstant('uActionsSize', tfInt, oEnvironment.uActionsSize)
            tuOne64 = tfConstant('uOne64', tf.int64, 1)

            if fMaxGradientNormal is not None:
                tfMaxGradientNormal = tfConstant('fMaxGradientNormal', tfFloat, fMaxGradientNormal)

            if self.uLogLevel > 1:
                tfGradientNormalUpdateCoef = tfConstant('fGradientNormalUpdateCoef', tfFloat, fGradientNormalUpdateCoef)
                tfGradientNormalForgetCoef = tfConstant('fGradientNormalForgetCoef', tfFloat, 1.0 - fGradientNormalUpdateCoef)

            if self.oPlayer.oEnvironment.bDiscrete:
                tfTotalMinLogInput = tfConstant('fMinClip', tfFloat, 1e-8)
                tfTotalMaxLogInput = tfConstant('fMaxClip', tfFloat, 1-1e-8)
            else:
                tfLogSqrtPi2 = tfConstant('fLogSqrtPi2', tfFloat, math.log(math.sqrt(math.pi * 2.0)))
                tfTotalMinScale = tfConstant('fMinScale', tfFloat, -20)
                tfTotalMaxScale = tfConstant('fMaxScale', tfFloat, 2)

            tfTrainableTargetAverageUpdateCoef = tfConstant('fTrainableTargetAverageUpdateCoef', tfFloat, fTrainableTargetAverageUpdateCoef)
            tfTrainableTargetAverageForgetCoef = tfConstant('fTrainableTargetAverageForgetCoef', tfFloat, 1.0 - fTrainableTargetAverageUpdateCoef)

        with tf1.variable_scope('Var', reuse=tf1.AUTO_REUSE):
            # Текущий эпизод
            self.tuEpisode = tfGlobalVariable('uEpisode', tfInt, 1)
            aTrainVariables.append(self.tuEpisode)

        # Тело генетического буфера повтора

        with tf.name_scope('ReplayBuffer'):
            with tf1.variable_scope('Const', reuse=tf1.AUTO_REUSE):
                # Вместительность циклического буфера
                self.tuReplayCapacity = tfConstant('uCapacity', tfInt, uReplayCapacity)
                # Указатель начала в циклическом буфере
                self.tuReplayStart = tfGlobalVariable('uStart', tfInt, 0)
                aReplayBufferVariables.append(self.tuReplayStart)
                # Указатель конца в циклическом буфере
                self.tuReplayEnd = tfGlobalVariable('uEnd', tfInt, 0)
                aReplayBufferVariables.append(self.tuReplayEnd)

                # Настройки древовидного буфера
                if self.bPriorityMode:
                    uTreeSize = int(math.pow(2, math.ceil(math.log2(uReplayCapacity))))
                    tuReplayTreeSize = tfConstant('uTreeSize', tfInt, uTreeSize)
                    tuHalfReplayTreeSize = tfConstant('uHalfTreeSize', tfInt, uTreeSize>>1)
                    tauOne = tfConstant('auOne', (tfInt, [1]), [1])

                    # Коэффициенты для вычисления приоритета
                    # `priority = clip(pow(error + eps, -power), min, max)`
                    fErrorPower = 0.6
                    fMinPriority = 0.1
                    fMaxPriority = 1.0

                    tfReplayErrorPower = tfConstant('fErrorPower', tfFloat, fErrorPower)
                    tfReplayMinPriority = tfConstant('fMinPriority', tfFloat, fMinPriority)
                    tfReplayMaxPriority = tfConstant('fMaxPriority', tfFloat, fMaxPriority)
                    tafReplayMaxPriority = tfConstant('afMaxPriority', (tfFloat, [1]), [fMaxPriority])
                    tfReplayPriorityEpsilon = tfConstant('fPriorityEpsilon', tfFloat, 0.01)

                    # Коэффициенты для вычисления веса на основе приоритета
                    fWeightPower = 0.4
                    uWeightFeedSteps = 2e5

                    tfReplayWeightDiff = tfConstant('fWeightDiff', tfFloat, (1.0 - fWeightPower) / float(uWeightFeedSteps))

            with tf1.variable_scope('Var', reuse=tf1.AUTO_REUSE):
                # Циклический буфер: наблюдения, действия, награды
                self.tafReplayPrevObservations = tfGlobalVariable('afPrevObservations', (tfFloat, [uReplayCapacity, self.oPlayer.oEnvironment.uObservationSize]), 0)
                aReplayBufferVariables.append(self.tafReplayPrevObservations)
                self.tafReplayObservations = tfGlobalVariable('afObservations', (tfFloat, [uReplayCapacity, self.oPlayer.oEnvironment.uObservationSize]), 0)
                aReplayBufferVariables.append(self.tafReplayObservations)
                if self.oPlayer.oEnvironment.bDiscrete:
                    self.tauReplayActions = tfGlobalVariable('auActions', (tfInt, [uReplayCapacity]), 0)
                    aReplayBufferVariables.append(self.tauReplayActions)
                else:
                    self.tafReplayActions = tfGlobalVariable('afActions', (tfFloat, [uReplayCapacity, self.oPlayer.oEnvironment.uActionsSize]), 0)
                    aReplayBufferVariables.append(self.tafReplayActions)
                if self.bEpisodeRating:
                    self.tafReplayRatings = tfGlobalVariable('afRatings', (tfFloat, [uReplayCapacity]), 0)
                    aReplayBufferVariables.append(self.tafReplayRatings)
                else:
                    self.tafReplayRewards = tfGlobalVariable('afRewards', (tfFloat, [uReplayCapacity]), 0)
                    aReplayBufferVariables.append(self.tafReplayRewards)
                self.tafReplayDones = tfGlobalVariable('afDones', (tfFloat, [uReplayCapacity]), 0)
                aReplayBufferVariables.append(self.tafReplayDones)

                # Древовидный буфер для быстрого поиска и случайной выборки с учётом приоритета
                if self.bPriorityMode:
                    # Степень значимости приоритетов (со временем стремится к 1)
                    self.tfReplayWeightPower = tfGlobalVariable('fWeightPower', tfFloat, fWeightPower)
                    aReplayBufferVariables.append(self.tfReplayWeightPower)

                    # Древовидный буфер максимумов
                    self.tafReplayMaxTree = tfGlobalVariable('afMaxTree', (tfFloat, [uTreeSize * 2]), 0)
                    aReplayBufferVariables.append(self.tafReplayMaxTree)
                    # Древовидный буфер сум
                    self.tafReplaySumTree = tfGlobalVariable('afSumTree', (tfFloat, [uTreeSize * 2]), 0)
                    aReplayBufferVariables.append(self.tafReplaySumTree)

            # Функциональный блок `fnAdd`, добавляющий записи в буфер повтора
            with tf.name_scope('fnAdd'):
                # Входные данные
                with tf1.variable_scope('Input', reuse=tf1.AUTO_REUSE):
                    self.tinafReplayPrevObservation = tfInput('afPrevObservation', (tfFloat, [self.oPlayer.oEnvironment.uObservationSize]))
                    self.tinafReplayObservation = tfInput('afObservation', (tfFloat, [self.oPlayer.oEnvironment.uObservationSize]))
                    if self.oPlayer.oEnvironment.bDiscrete:
                        self.tinauReplayAction = tfInput('auAction', (tfInt, [1]))
                    else:
                        self.tinafReplayActions = tfInput('afActions', (tfFloat, [self.oPlayer.oEnvironment.uActionsSize]))
                    if self.bEpisodeRating:
                        self.tinafReplayRating = tfInput('afRating', (tfFloat, [1]))
                    else:
                        self.tinafReplayReward = tfInput('afReward', (tfFloat, [1]))
                    self.tinfReplayDone = tfInput('fDone', tfFloat)
                    self.tinfReplayScore = tfInput('fScore', tfFloat)
                    self.tinuReplaySteps = tfInput('uSteps', tfInt)

                tafDones = tf.expand_dims(self.tinfReplayDone, axis=-1)

                # Индексный лимит
                tuMaxIndex = self.tuReplayStart + self.tuReplayCapacity

                # Сохранить входные данные в буфере
                tuIndex = tf.math.floormod(self.tuReplayEnd, self.tuReplayCapacity)
                tauIndices = tf.expand_dims(tf.expand_dims(tuIndex, axis=-1), axis=-1)
                topUpdate1 = self.tafReplayPrevObservations.scatter_nd_update(tauIndices, self.tinafReplayPrevObservation[None, :])
                topUpdate2 = self.tafReplayObservations.scatter_nd_update(tauIndices, self.tinafReplayObservation[None, :])
                if oPlayer.oEnvironment.bDiscrete:
                    topUpdate3 = self.tauReplayActions.scatter_nd_update(tauIndices, self.tinauReplayAction)
                else:
                    topUpdate3 = self.tafReplayActions.scatter_nd_update(tauIndices, self.tinafReplayActions[None, :])
                if self.bEpisodeRating:
                    topUpdate4 = self.tafReplayRatings.scatter_nd_update(tauIndices, self.tinafReplayRating)
                else:
                    topUpdate4 = self.tafReplayRewards.scatter_nd_update(tauIndices, self.tinafReplayReward)
                topUpdate5 = self.tafReplayDones.scatter_nd_update(tauIndices, tafDones)

                with tfWait([topUpdate1, topUpdate2, topUpdate3, topUpdate4, topUpdate5]):
                    tuNewEnd = self.tuReplayEnd.assign_add(tuOne)

                with tfWait([tuNewEnd]):
                    # Проверить буфер на переполнение
                    def fnOverflow():
                        return self.tuReplayStart.assign(self.tuReplayEnd + tuOne - self.tuReplayCapacity)
                    def fnNoOverflow():
                        return self.tuReplayStart
                    tuNewStart = tf.cond(tf.greater(self.tuReplayEnd, tuMaxIndex), fnOverflow, fnNoOverflow, 'uNewStart')

                aWaitList = [tuNewStart]

                # Логировать конечный счёт эпизода
                if self.uLogLevel > 0:
                    def fnAddLogEpisode():
                        with self.oSummaryWriter.as_default(): # pylint: disable=not-context-manager
                            topLogScore = tf.summary.scalar('Stats/Score', self.tinfReplayScore, tf.cast(self.tuEpisode, tf.int64))
                            if self.uLogLevel > 1:
                                topLogSteps = tf.summary.scalar('Info/Steps', self.tinuReplaySteps, tf.cast(self.tuEpisode, tf.int64))
                                topLog = tf.group(topLogScore, topLogSteps)
                            else:
                                topLog = topLogScore
                        return topLog
                    def fnAddLogStep():
                        return tf.no_op()
                    aWaitList.append(tf.cond(tf.equal(self.tinfReplayDone, tfZero), true_fn=fnAddLogStep, false_fn=fnAddLogEpisode))

                with tfWait(aWaitList):
                    # Перейти к следующему эпизоду
                    def fnNextEpisode():
                        return self.tuEpisode.assign_add(tuOne)
                    def fnCurEpisode():
                        return self.tuEpisode
                    tuNewEpisode = tf.cond(tf.not_equal(self.tinfReplayDone, tfZero), fnNextEpisode, fnCurEpisode, 'uNewEpisode')

                # Обновить приоритеты в древовидном буфере
                if self.bPriorityMode:
                    tuIndex = tuIndex + tuReplayTreeSize
                    tauIndices = tf.expand_dims(tf.expand_dims(tuIndex, axis=-1), axis=-1)
                    topUpdateMax = self.tafReplayMaxTree.scatter_nd_update(tauIndices, tafReplayMaxPriority)
                    topUpdateSum = self.tafReplaySumTree.scatter_nd_update(tauIndices, tafReplayMaxPriority)
                    tuIndex = tf.math.floordiv(tuIndex, tuTwo)
                    with tfWait([topUpdateMax, topUpdateSum]):
                        def fnAddCompare(tuLoopIndex):
                            return tf.greater_equal(tuLoopIndex, tuOne)
                        def fnAddLoop(tuLoopIndex):
                            tauLeft = tf.expand_dims(tuLoopIndex * tuTwo, axis=-1)
                            tauRight = tauLeft + tuOne
                            tfMax = tf.maximum(tf.gather(self.tafReplayMaxTree, tauLeft), tf.gather(self.tafReplayMaxTree, tauRight)) # pylint: disable=no-value-for-parameter
                            tfSum = tf.add(tf.gather(self.tafReplaySumTree, tauLeft), tf.gather(self.tafReplaySumTree, tauRight)) # pylint: disable=no-value-for-parameter
                            tauIndices = tf.expand_dims(tf.expand_dims(tuLoopIndex, axis=-1), axis=-1)
                            topUpdateMax = self.tafReplayMaxTree.scatter_nd_update(tauIndices, tfMax)
                            topUpdateSum = self.tafReplaySumTree.scatter_nd_update(tauIndices, tfSum)
                            with tfWait([topUpdateMax, topUpdateSum]):
                                tuLoopIndex = tf.math.floordiv(tuLoopIndex, tuTwo)
                                return [tuLoopIndex]
                        [topUpdateTree] = tf.while_loop(fnAddCompare, fnAddLoop, [tuIndex])

            # Узел результата функции добавления данных в буфер повтора
            if self.bPriorityMode:
                self.tfnReplayAdd = tf.group(tuNewEpisode, topUpdateTree)
            else:
                self.tfnReplayAdd = tuNewEpisode

            # Функциональный блок `fnReadBatch`, считывающий блок записей из буфера повтора
            with tf.name_scope('fnReadBatch'):
                # Создать блок индексов для выборки из буфера повтора
                if self.bPriorityMode:
                    # Общая сумма всех приоритетов
                    tfTotalPriority = tf.squeeze(tf.gather(self.tafReplaySumTree, tauOne)) # pylint: disable=no-value-for-parameter
                    # Набор случайных смещений приоритетов
                    tafRandomPriorities = tf.random.uniform([uBatchSize], minval=tfZero, maxval=tfTotalPriority, dtype=tfFloat, seed=None) # pylint: disable=unexpected-keyword-arg
                    # Найти соответствующие смещениям приоритетов индексы
                    tuIndex = tuOne
                    tauIndices = tf.ones([uBatchSize], dtype=tfInt)
                    def fnRandomCompare(tuLoopIndex, tauIndices, tafRandomPriorities):
                        return tf.less(tuLoopIndex, tuReplayTreeSize)
                    def fnRandomLoop(tuLoopIndex, tauIndices, tafRandomPriorities):
                        tauLeft = tauIndices * tuTwo
                        tafValues = tf.gather(self.tafReplaySumTree, tauLeft) # pylint: disable=no-value-for-parameter
                        tabLessEqual = tf.less_equal(tafValues, tafRandomPriorities)
                        return [tuLoopIndex * tuTwo, tauLeft + tf.cast(tabLessEqual, tfInt), tafRandomPriorities - tafValues * tf.cast(tabLessEqual, tfFloat)]
                    [tuReturnIndex, tauReplayIndices, tafReturnRandomPriorities] = tf.while_loop(fnRandomCompare, fnRandomLoop, [tuIndex, tauIndices, tafRandomPriorities])
                    # Расчитать весовые Коэффициенты для каждого индекса на основе его приоритета
                    tfMaxPriorityScale = tfOne / tf.squeeze(tf.gather(self.tafReplayMaxTree, tauOne)) # pylint: disable=no-value-for-parameter
                    tafBatchPriorities = tf.gather(self.tafReplaySumTree, tauReplayIndices) # pylint: disable=no-value-for-parameter
                    tafBatchWeights = tf.pow(tafBatchPriorities * tfMaxPriorityScale, self.tfReplayWeightPower) # pylint: disable=invalid-unary-operand-type
                    # Обновить коэффициент влияющий на веса
                    topUpdateReplayWeightPower = self.tfReplayWeightPower.assign(tf.minimum(tfOne, self.tfReplayWeightPower + tfReplayWeightDiff))

                    with tfWait([tuReturnIndex, tauReplayIndices, tafReturnRandomPriorities, topUpdateReplayWeightPower]):
                        # Набор случайных индексов
                        tauRandomIndices = tauReplayIndices - tuReplayTreeSize
                else:
                    # Набор случайных смещений
                    tauRandomOffsets = tf.random.uniform([uBatchSize], minval=self.tuReplayStart, maxval=self.tuReplayEnd, dtype=tfInt, seed=None) # pylint: disable=unexpected-keyword-arg
                    # Набор случайных индексов
                    tauRandomIndices = tf.math.floormod(tauRandomOffsets, self.tuReplayCapacity)

                # Создать блок данных на основе выборки по случайным индексам
                tafBatchPrevObservations = tf.gather(self.tafReplayPrevObservations, tauRandomIndices) # pylint: disable=no-value-for-parameter
                tafBatchObservations = tf.gather(self.tafReplayObservations, tauRandomIndices) # pylint: disable=no-value-for-parameter
                if self.oPlayer.oEnvironment.bDiscrete:
                    tauBatchActions = tf.gather(self.tauReplayActions, tauRandomIndices) # pylint: disable=no-value-for-parameter
                else:
                    tafBatchActions = tf.gather(self.tafReplayActions, tauRandomIndices) # pylint: disable=no-value-for-parameter
                if self.bEpisodeRating:
                    tafBatchRatings = tf.gather(self.tafReplayRatings, tauRandomIndices) # pylint: disable=no-value-for-parameter
                else:
                    tafBatchRewards = tf.gather(self.tafReplayRewards, tauRandomIndices) # pylint: disable=no-value-for-parameter
                tafBatchDones = tf.gather(self.tafReplayDones, tauRandomIndices) # pylint: disable=no-value-for-parameter

        # Тело агента `Soft Actor Critic`

        with tf.name_scope('SacAgent'):
            with tf.name_scope('Critic'):
                # Создать две нейросети `Критик-тренер` и две нейросети `Критик-цель`
                self.annCriticNetworks = [CriticNetwork() for _ in range(4)]
                self.nnTrainCritic1 = self.annCriticNetworks[0]
                self.nnTrainCritic2 = self.annCriticNetworks[1]
                self.nnTargetCritic1 = self.annCriticNetworks[2]
                self.nnTargetCritic2 = self.annCriticNetworks[3]
                [nnCritic.build((self.uBatchSize, oEnvironment.uObservationSize), (self.uBatchSize, oEnvironment.uActionsSize)) for nnCritic in self.annCriticNetworks]
                [aTrainVariables.extend(nnCritic.trainable_variables) for nnCritic in self.annCriticNetworks]

            with tf.name_scope('Actor'):
                # Создать нейросеть `Актёр-тренер`
                self.nnTrainActor = CreateActor(oEnvironment, self.uBatchSize, 'nnSacTrainActor')
                aTrainVariables.extend(self.nnTrainActor.trainable_variables)

                if not self.oPlayer.oEnvironment.bDiscrete:
                    afActionsMin = np.array([0])
                    afActionsMax = np.array([1])

                    # Константы непрерывной модели
                    with tf1.variable_scope('Const', reuse=tf1.AUTO_REUSE):
                        tafActionsMean = tfConstant('afActionsMean', tfFloat, (afActionsMin + afActionsMax) / 2)
                        tafActionsStd = tfConstant('afActionsStd', tfFloat, (afActionsMin - afActionsMax) / 2)

            with tf1.variable_scope('Var', reuse=tf1.AUTO_REUSE):
                # `Альфа-регулятор` 
                self.tfLogAlpha = tfGlobalVariable('fLogAlpha', tfFloat, 0, True)
                aTrainVariables.append(self.tfLogAlpha)

                if self.uLogLevel > 1:
                    tfCriticGradientAverageNormal = tfGlobalVariable('fCriticGradientClip', tfFloat, 0)
                    aTrainVariables.append(tfCriticGradientAverageNormal)
                    tfActorGradientAverageNormal = tfGlobalVariable('fActorGradientClip', tfFloat, 0)
                    aTrainVariables.append(tfActorGradientAverageNormal)

                if tfTrainStepCounter is None:
                    tfTrainStepCounter = tf.compat.v1.train.get_or_create_global_step()
                aTrainVariables.append(tfTrainStepCounter)

            with tf1.variable_scope('Const', reuse=tf1.AUTO_REUSE):
                # Фактор забывания оценки
                tfRatingDiscountFactor = tfConstant('fRatingDiscountFactor', tfFloat, self.fRatingDiscountFactor)
                # Желаемая энтропия
                if self.oPlayer.oEnvironment.bDiscrete:
                    tfTargetEntropy = tfConstant('fTargetEntropy', tfFloat, -np.log(1.0 / oEnvironment.uActionsSize) * 0.98)
                else:
                    tfTargetEntropy = tfConstant('fTargetEntropy', tfFloat, oEnvironment.uActionsSize / 2.0)

            # Функциональный блок `fnInitialize`, производящий инициализацию всех переменных при первом запуске
            with tf.name_scope('fnInitialize'):
                topCriticUpdate1 = fnHardUpdate(self.nnTargetCritic1.variables, self.nnTrainCritic1.variables)
                topCriticUpdate2 = fnHardUpdate(self.nnTargetCritic2.variables, self.nnTrainCritic2.variables)
                topActorUpdate = fnHardUpdate(self.oPlayer.nnActor.variables, self.nnTrainActor.variables)
            # Узел результата функции инициализации
            self.tfnInitialize = tf.group(topCriticUpdate1, topCriticUpdate2, topActorUpdate)

            # Обновить целевые нейронные сети
            def fnUpdateTarget():
                topCriticUpdate1 = fnSoftUpdate(self.nnTargetCritic1.variables, self.nnTrainCritic1.variables, tfZero, tfOne, tfTrainableTargetAverageForgetCoef, tfTrainableTargetAverageUpdateCoef)
                topCriticUpdate2 = fnSoftUpdate(self.nnTargetCritic2.variables, self.nnTrainCritic2.variables, tfZero, tfOne, tfTrainableTargetAverageForgetCoef, tfTrainableTargetAverageUpdateCoef)
                topActorUpdate = fnHardUpdate(self.oPlayer.nnActor.variables, self.nnTrainActor.variables)
                return tf.group(topCriticUpdate1, topCriticUpdate2, topActorUpdate)

            # Функциональный блок `fnTrain`, выполняющий все операции, связанные с обучением сетей
            with tf.name_scope('fnTrain'):
                # Форматировать входные данные
                if self.oPlayer.oEnvironment.bDiscrete:
                    tafActionsOneHots = tf.one_hot(tauBatchActions, tuActionsSize, on_value=tfOne, off_value=tfZero, dtype=tfFloat) # pylint: disable=unexpected-keyword-arg
                    tafStates = self.nnTrainCritic1.prepare(tafBatchPrevObservations, tafActionsOneHots)
                else:
                    tafStates = self.nnTrainCritic1.prepare(tafBatchPrevObservations, tafBatchActions)

                # Рассчитать рейтинги для входных данных
                if self.bEpisodeRating:
                    # Для эпизодического режима, рейтинги уже рассчитаны заранее
                    tafRatings = tafBatchRatings
                else:
                    # Рассчитать вероятности следующих действий
                    if self.oPlayer.oEnvironment.bDiscrete:
                        # Ненормализованные предполагаемые логарифмические вероятности возможных действий
                        tafNextUnnormalizedLogProbabilities = self.nnTrainActor.call(tafBatchObservations)
                        # Массив случайных чисел для добавления шума к логарифмическим вероятностям
                        tafRandomUniforms = tf.random.uniform([self.uBatchSize, oEnvironment.uActionsSize], minval=tfTotalMinLogInput, maxval=tfTotalMaxLogInput, dtype=tfFloat, seed=None) # pylint: disable=unexpected-keyword-arg
                        # Распределение Гумбеля для случайных чисел
                        tafGumbels = -tf.math.log(-tf.math.log(tafRandomUniforms)) # pylint: disable=invalid-unary-operand-type
                        # Ненормализованные предполагаемые логарифмические вероятности возможных действий с добавлением шума
                        tafNextUnnormalizedNoisyLogProbabilities = tafNextUnnormalizedLogProbabilities + tafGumbels
                        # Наилучшие предполагаемые действия
                        tauNextBestActions = tf.argmax(tafNextUnnormalizedNoisyLogProbabilities, axis=-1, output_type=tfInt)
                        # Вектора наилучших предполагаемых действий
                        tafNextPredActions = tf.one_hot(tauNextBestActions, tuActionsSize, on_value=tfOne, off_value=tfZero, dtype=tfFloat) # pylint: disable=unexpected-keyword-arg
                        # Нормализованные предполагаемые логарифмические вероятности возможных действий
                        tafNextNormalizedLogProbabilities = tf.math.log_softmax(tafNextUnnormalizedLogProbabilities, axis=-1)
                        # Логарифмические вероятности наилучших предполагаемых действий (в роли энтропии)
                        tafNextProbabilitiesEntropy = -tf.math.reduce_sum(tafNextPredActions * tafNextNormalizedLogProbabilities, axis=-1)
                    else:
                        tafNextLocations, tafNextScales = self.nnTrainActor.call(tafBatchObservations)
                        tafNextClippedScales = tf.clip_by_value(tafNextScales, tfTotalMinScale, tfTotalMaxScale)
                        tafNextClippedScalesExp = tf.math.exp(tafNextClippedScales)			
                        tafNextRandomNormals = tf.random.normal([self.uBatchSize, oEnvironment.uActionsSize], mean=tafNextLocations, stddev=tafNextClippedScalesExp, dtype=tfFloat, seed=None)
                        tafNextPredActions = tf.math.tanh(tafNextRandomNormals) * tafActionsStd + tafActionsMean
                        tafNextClippedScalesExpSquare = tf.square(tafNextClippedScalesExp)
                        tafNextProbabilitiesEntropy = (tf.square(tafNextPredActions - tafNextLocations)) / (2 * tafNextClippedScalesExpSquare) + tafNextClippedScales + tfLogSqrtPi2 # pylint: disable=invalid-unary-operand-type

                    # Рассчитать следующие вероятные рейтинги
                    tafNextStates = self.nnTargetCritic1.prepare(tafBatchObservations, tafNextPredActions)
                    tafPredNextRatings1 = tf.squeeze(self.nnTargetCritic1.call(tafNextStates), axis=-1)
                    tafPredNextRatings2 = tf.squeeze(self.nnTargetCritic2.call(tafNextStates), axis=-1)
                    tafPredNextRatings = tf.minimum(tafPredNextRatings1, tafPredNextRatings2) + tf.math.exp(self.tfLogAlpha) * tafNextProbabilitiesEntropy

                    # Рассчитать рейтинги
                    tafRatings = tafBatchRewards + (1. - tafBatchDones) * tafPredNextRatings * tfRatingDiscountFactor

                # Вычислить ошибку обучения нейросети `Критик-тренер` #1
                tfaTrainableCriticVariables1 = self.nnTrainCritic1.trainable_variables
                with tf.GradientTape(watch_accessed_variables=False) as tape:
                    tape.watch(tfaTrainableCriticVariables1)

                    # Предполагаемые рейтинги
                    tafPredRatings1 = tf.squeeze(self.nnTrainCritic1.call(tf.stop_gradient(tafStates)), axis=-1)
                    # Приимущество реальых рейтингов перед предугаданными
                    tafAdvantages1 = tf.stop_gradient(tafRatings) - tafPredRatings1
                    # Ошибка обучения
                    if self.bPriorityMode:
                        tfCriticLoss1 = tf.reduce_mean(tf.square(tafAdvantages1) * tf.stop_gradient(tafBatchWeights))
                    else:
                        tfCriticLoss1 = tf.reduce_mean(tf.square(tafAdvantages1))

                # Вычислить градиенты
                aCriticGradients1 = tape.gradient(tfCriticLoss1, tfaTrainableCriticVariables1)

                # Вычислить ошибку обучения нейросети `Критик-тренер` #2
                tfaTrainableCriticVariables2 = self.nnTrainCritic2.trainable_variables
                with tf.GradientTape(watch_accessed_variables=False) as tape:
                    tape.watch(tfaTrainableCriticVariables2)

                    # Предполагаемые рейтинги
                    tafPredRatings2 = tf.squeeze(self.nnTrainCritic2.call(tf.stop_gradient(tafStates)), axis=-1)
                    # Приимущество реальых рейтингов перед предугаданными
                    tafAdvantages2 = tf.stop_gradient(tafRatings) - tafPredRatings2
                    # Ошибка обучения
                    if self.bPriorityMode:
                        tfCriticLoss2 = tf.reduce_mean(tf.square(tafAdvantages2) * tf.stop_gradient(tafBatchWeights))
                    else:
                        tfCriticLoss2 = tf.reduce_mean(tf.square(tafAdvantages2))

                # Вычислить градиенты
                aCriticGradients2 = tape.gradient(tfCriticLoss2, tfaTrainableCriticVariables2)

                # Общая ошибка критика
                tfCriticLoss = (tfCriticLoss1 + tfCriticLoss2) * tfHalf

                # Рассчитать усреднённое нормальное значение градентов `Критиков-тренеров`
                if self.uLogLevel > 1:
                    tuGradientsCount = tfZero
                    tfGradientsNormalMass = tfZero
                    for tfaGradientsGroup in aCriticGradients1:
                        tfaGradients = tfaGradientsGroup.values if isinstance(tfaGradientsGroup, tf.IndexedSlices) else tfaGradientsGroup
                        tuCount = tf.cast(tf.size(tfaGradients), dtype=tfFloat) # pylint: disable=unexpected-keyword-arg, no-value-for-parameter
                        tuGradientsCount += tuCount
                        tfGradientsNormalMass += tf.linalg.global_norm([tfaGradients]) * tuCount
                    for tfaGradientsGroup in aCriticGradients2:
                        tfaGradients = tfaGradientsGroup.values if isinstance(tfaGradientsGroup, tf.IndexedSlices) else tfaGradientsGroup
                        tuCount = tf.cast(tf.size(tfaGradients), dtype=tfFloat) # pylint: disable=unexpected-keyword-arg, no-value-for-parameter
                        tuGradientsCount += tuCount
                        tfGradientsNormalMass += tf.linalg.global_norm([tfaGradients]) * tuCount
                    tfCriticGradientNormal = tfCriticGradientAverageNormal.assign(tfCriticGradientAverageNormal * tfGradientNormalForgetCoef + (tfGradientsNormalMass / tuGradientsCount) * tfGradientNormalUpdateCoef)

                # Обрезать значения градиентов для предотвращения ошибок `inf` и `nan`
                if fMaxGradientNormal is not None:
                    aCriticGradients1 = fnClipGradients(aCriticGradients1, tfMaxGradientNormal)
                    aCriticGradients2 = fnClipGradients(aCriticGradients2, tfMaxGradientNormal)

                # Обновить приоритеты в буфере повтора
                if self.bPriorityMode:
                    # Вычислить приоритеты на основе приимущества рейтингов
                    tafReplayErrors = tf.minimum(tf.abs(tafAdvantages1), tf.abs(tafAdvantages2))
                    tafReplayPriorities = tf.clip_by_value(tf.pow(tafReplayErrors + tfReplayPriorityEpsilon, -tfReplayErrorPower), tfReplayMinPriority, tfReplayMaxPriority)
                    # Обновить приоритеты
                    tauUpdateIndices = tf.expand_dims(tauReplayIndices, axis=-1)
                    topUpdateMax = self.tafReplayMaxTree.scatter_nd_update(tauUpdateIndices, tafReplayPriorities)
                    topUpdateSum = self.tafReplaySumTree.scatter_nd_update(tauUpdateIndices, tafReplayPriorities)
                    with tfWait([topUpdateMax, topUpdateSum]):
                        tuIndex = tuHalfReplayTreeSize
                        tauIndices = tf.math.floordiv(tauReplayIndices, tuTwo)
                        def fnUpdateCompare(tuLoopIndex, tauIndices):
                            return tf.greater_equal(tuLoopIndex, tuOne)
                        def fnUpdateLoop(tuLoopIndex, tauIndices):
                            tauLeft = tauIndices * tuTwo
                            tauRight = tauLeft + tuOne
                            tfMax = tf.maximum(tf.gather(self.tafReplayMaxTree, tauLeft), tf.gather(self.tafReplayMaxTree, tauRight)) # pylint: disable=no-value-for-parameter
                            tfSum = tf.add(tf.gather(self.tafReplaySumTree, tauLeft), tf.gather(self.tafReplaySumTree, tauRight)) # pylint: disable=no-value-for-parameter
                            tauUpdateIndices = tf.expand_dims(tauIndices, axis=-1)
                            topUpdateMax = self.tafReplayMaxTree.scatter_nd_update(tauUpdateIndices, tfMax)
                            topUpdateSum = self.tafReplaySumTree.scatter_nd_update(tauUpdateIndices, tfSum)
                            with tfWait([topUpdateMax, topUpdateSum]):
                                tauIndices = tf.math.floordiv(tauIndices, tuTwo)
                                tuLoopIndex = tf.math.floordiv(tuLoopIndex, tuTwo)
                                return [tuLoopIndex, tauIndices]
                        [tuIndex, tauIndices] = tf.while_loop(fnUpdateCompare, fnUpdateLoop, [tuIndex, tauIndices])
                        topUpdateReplayPriorities = tf.group(tuIndex, tauIndices)

                # Вычислить ошибку обучения нейросети `Актёр-тренер`
                tfaTrainableActorVariables = self.nnTrainActor.trainable_variables
                with tf.GradientTape(watch_accessed_variables=False) as tape:
                    tape.watch(tfaTrainableActorVariables)

                    # Рассчитать вероятности всех возможных действий
                    if self.oPlayer.oEnvironment.bDiscrete:
                        # Ненормализованные предполагаемые логарифмические вероятности действий
                        tafUnnormalizedLogProbabilities = self.nnTrainActor.call(tf.stop_gradient(tafBatchPrevObservations))
                        # Массив случайных чисел для добавления шума к логарифмическим вероятностям
                        tafRandomUniforms = tf.random.uniform([self.uBatchSize, oEnvironment.uActionsSize], minval=tfTotalMinLogInput, maxval=tfTotalMaxLogInput, dtype=tfFloat, seed=None) # pylint: disable=unexpected-keyword-arg
                        # Распределение Гумбеля для случайных чисел
                        tafGumbels = -tf.math.log(-tf.math.log(tafRandomUniforms)) # pylint: disable=invalid-unary-operand-type
                        # Ненормализованные предполагаемые логарифмические вероятности возможных действий с добавлением шума
                        tafUnnormalizedNoisyLogProbabilities = tafUnnormalizedLogProbabilities + tafGumbels
                        # Вектора предполагаемых действий
                        tafPredActions = tf.math.exp(tafUnnormalizedNoisyLogProbabilities - tf.math.reduce_logsumexp(tafUnnormalizedNoisyLogProbabilities, axis=-1, keepdims=True))
                        # Нормализованные предполагаемые логарифмические вероятности возможных действий
                        tafPredNormalizedLogProbabilities = tf.math.log_softmax(tafUnnormalizedLogProbabilities, axis=-1)
                        # Логарифмические вероятности наилучших предполагаемых действий (в роли энтропии)
                        tafPredProbabilitiesEntropy = -tf.math.reduce_sum(tafPredActions * tafPredNormalizedLogProbabilities, axis=-1)
                    else:
                        tafLocations, tafScales = self.nnTrainActor.call(tf.stop_gradient(tafBatchPrevObservations))
                        tafClippedScales = tf.clip_by_value(tafScales, tfTotalMinScale, tfTotalMaxScale)
                        tafClippedScalesExp = tf.math.exp(tafClippedScales)
                        tafRandomNormals = tf.random.normal([self.uBatchSize,oEnvironment.uActionsSize], mean=tfZero, stddev=tfOne, dtype=tfFloat, seed=None)
                        tafRandomUnscaledActions = tafLocations + tafClippedScalesExp * tafRandomNormals
                        tafPredActions = tf.math.tanh(tafRandomUnscaledActions) * tafActionsStd + tafActionsMean
                        tafClippedScalesExpPower = tf.math.square(tafClippedScalesExp)
                        tafNormalizers = -tf.math.reduce_sum(tf.math.log(1 - tf.math.square(tafPredActions) + 1e-6), axis=1)
                        tafPredProbabilitiesEntropy = (tf.math.square(tafRandomUnscaledActions - tafLocations)) / (2 * tafClippedScalesExpPower) + tafClippedScales + tfLogSqrtPi2 + tafNormalizers # pylint: disable=invalid-unary-operand-type

                    # Рассчитать предполагаемые вероятные рейтинги
                    tafRandomStates = self.nnTrainCritic1.prepare(tf.stop_gradient(tafBatchPrevObservations), tafPredActions)
                    tafRandomPredRatings1 = tf.squeeze(self.nnTrainCritic1.call(tafRandomStates), axis=-1)
                    tafRandomPredRatings2 = tf.squeeze(self.nnTrainCritic2.call(tafRandomStates), axis=-1)
                    tafRandomPredRatings = tf.minimum(tafRandomPredRatings1, tafRandomPredRatings2) + tf.exp(self.tfLogAlpha) * tafPredProbabilitiesEntropy

                    # Ошибка обучения
                    if self.bPriorityMode:
                        tfActorLoss = -tf.math.reduce_mean(tafRandomPredRatings * tf.stop_gradient(tafBatchWeights))
                    else:
                        tfActorLoss = -tf.math.reduce_mean(tafRandomPredRatings)

                # Вычислить градиенты
                aActorGradients = tape.gradient(tfActorLoss, tfaTrainableActorVariables)

                # Рассчитать усреднённое нормальное значение градентов `Актёра-тренера`
                if self.uLogLevel > 1:
                    tuGradientsCount = tfZero
                    tfGradientsNormalMass = tfZero
                    for tfaGradientsGroup in aActorGradients:
                        tfaGradients = tfaGradientsGroup.values if isinstance(tfaGradientsGroup, tf.IndexedSlices) else tfaGradientsGroup
                        tuCount = tf.cast(tf.size(tfaGradients), dtype=tfFloat) # pylint: disable=unexpected-keyword-arg, no-value-for-parameter
                        tuGradientsCount += tuCount
                        tfGradientsNormalMass += tf.linalg.global_norm([tfaGradients]) * tuCount
                    tfActorGradientNormal = tfActorGradientAverageNormal.assign(tfActorGradientAverageNormal * tfGradientNormalForgetCoef + (tfGradientsNormalMass / tuGradientsCount) * tfGradientNormalUpdateCoef)

                # Обрезать значения градиентов для предотвращения ошибок `inf` и `nan`
                if fMaxGradientNormal is not None:
                    aActorGradients = fnClipGradients(aActorGradients, tfMaxGradientNormal)

                # Вычислить ошибку обучения `Альфа-регулятора`
                tfaTrainableAlphaVariable = [self.tfLogAlpha]
                with tf.GradientTape(watch_accessed_variables=False) as tape:
                    tape.watch(tfaTrainableAlphaVariable)

                    # Вычислить ошибку энтропии
                    tafEntropyLoss = -(tafPredProbabilitiesEntropy - tfTargetEntropy)

                    # Ошибка обучения
                    if self.bPriorityMode:
                        tfAlphaLoss = tf.math.reduce_mean(tafEntropyLoss * tf.stop_gradient(tafBatchWeights)) * self.tfLogAlpha
                    else:
                        tfAlphaLoss = tf.math.reduce_mean(tafEntropyLoss) * self.tfLogAlpha

                # Вычислить градиенты
                aAlphaGradient = tape.gradient(tfAlphaLoss, tfaTrainableAlphaVariable)

                # Обрезать значения градиентов для предотвращения ошибок `inf` и `nan`
                if fMaxGradientNormal is not None:
                    aAlphaGradient = fnClipGradients(aAlphaGradient, tfMaxGradientNormal)

                # Обучение
                with tfWait([tfCriticLoss, tfActorLoss, tfAlphaLoss]):
                    topOptimizeCritic1 = self.koCriticOptimizer.apply_gradients(zip(aCriticGradients1, tfaTrainableCriticVariables1))
                    topOptimizeCritic2 = self.koCriticOptimizer.apply_gradients(zip(aCriticGradients2, tfaTrainableCriticVariables2))
                    topOptimizeActor = self.koActorOptimizer.apply_gradients(zip(aActorGradients, tfaTrainableActorVariables))
                    topOptimizeAlpha = self.koAlphaOptimizer.apply_gradients(zip(aAlphaGradient, tfaTrainableAlphaVariable))

                aWaitList = [topOptimizeCritic1, topOptimizeCritic2, topOptimizeActor, topOptimizeAlpha]

                if self.bPriorityMode:
                    aWaitList.append(topUpdateReplayPriorities)

        # Функциональный субблок логирования, выполняющий все операции, связанные со сбором статистики

        if self.uLogLevel > 0:
            with self.oSummaryWriter.as_default(): # pylint: disable=not-context-manager
                aWaitList.append(tf.summary.scalar('Stats/Loss/Critic', tf.reduce_mean(tfCriticLoss), tfTrainStepCounter))
                aWaitList.append(tf.summary.scalar('Stats/Loss/Actor', tf.reduce_mean(tfActorLoss), tfTrainStepCounter))
                aWaitList.append(tf.summary.scalar('Stats/Loss/Alpha', tfAlphaLoss, tfTrainStepCounter))

        if self.uLogLevel > 1:
            with self.oSummaryWriter.as_default(): # pylint: disable=not-context-manager
                aWaitList.append(tf.summary.scalar('Info/GradientNormal/Critic', tfCriticGradientNormal, tfTrainStepCounter))
                aWaitList.append(tf.summary.scalar('Info/GradientNormal/Actor', tfActorGradientNormal, tfTrainStepCounter))
                if self.bPriorityMode:
                    aWaitList.append(tf.summary.scalar('Info/LossWeight/Mean', tf.reduce_mean(tafBatchWeights), tfTrainStepCounter))

                aWaitList.append(tf.summary.scalar('Rating/LogAlpha', self.tfLogAlpha, tfTrainStepCounter))
                aWaitList.append(tf.summary.scalar('Rating/Alpha', tf.math.exp(self.tfLogAlpha), tfTrainStepCounter))
                aWaitList.append(tf.summary.scalar('Rating/Value/Mean', tf.reduce_mean(tafRatings), tfTrainStepCounter))
                aWaitList.append(tf.summary.scalar('Rating/Entropy/Mean', tf.reduce_mean(tafPredProbabilitiesEntropy), tfTrainStepCounter))

                aWaitList.append(tf.summary.histogram('Rating/Value', tafRatings, tfTrainStepCounter))
                aWaitList.append(tf.summary.histogram('Rating/Entropy', tafPredProbabilitiesEntropy, tfTrainStepCounter))
                aWaitList.append(tf.summary.histogram('Rating/Pred', tafRandomPredRatings, tfTrainStepCounter))
                tafAdvantages = tf.concat([tafAdvantages1, tafAdvantages2], 0)
                aWaitList.append(tf.summary.histogram('Rating/Advantage', tafAdvantages, tfTrainStepCounter))

        if self.uLogLevel > 2:
            aWaitList.append(fnWeightsSummary(self.oSummaryWriter, zip(aCriticGradients1, tfaTrainableCriticVariables1), tfTrainStepCounter))
            aWaitList.append(fnWeightsSummary(self.oSummaryWriter, zip(aCriticGradients2, tfaTrainableCriticVariables2), tfTrainStepCounter))
            aWaitList.append(fnWeightsSummary(self.oSummaryWriter, zip(aActorGradients, tfaTrainableActorVariables), tfTrainStepCounter))
            aWaitList.append(fnWeightsSummary(self.oSummaryWriter, zip(aAlphaGradient, tfaTrainableAlphaVariable), tfTrainStepCounter))

        with tf.name_scope('SacAgent'):
            # Продолжение... Функциональный блок `fnTrain`, выполняющий все операции, связанные с обучением сетей
            with tf.name_scope('fnTrain'):
                with tfWait(aWaitList):
                    # Переход к следующему шагу
                    topNewStepCounter = tfTrainStepCounter.assign_add(tuOne64)
                    # Обновить целевые нейросети используя метод `скользящего окна`
                    topUpdateTarget = fnUpdateTarget()

                with tfWait([topNewStepCounter, topUpdateTarget]):
                    # Узел результата функции тренировки
                    self.tfnTrain = tfCriticLoss + tfActorLoss + tfAlphaLoss

        # Сформировать папки для хранения модели

        self.sTargetRestorePath = sRestorePath + 'Target/'
        self.sTrainRestorePath = sRestorePath + 'Train/'
        self.sReplayBufferRestorePath = sRestorePath + 'ReplayBuffer/'

        self.oTargetSaver = tf.compat.v1.train.Saver(aTargetVariables, save_relative_paths=True)
        self.oTrainSaver = tf.compat.v1.train.Saver(aTrainVariables, save_relative_paths=True)
        self.oReplayBufferSaver = tf.compat.v1.train.Saver(aReplayBufferVariables, save_relative_paths=True)

        if not os.path.exists(self.sTargetRestorePath):
            os.makedirs(self.sTargetRestorePath, exist_ok=True)
        if not os.path.exists(self.sTrainRestorePath):
            os.makedirs(self.sTrainRestorePath, exist_ok=True)
        if not os.path.exists(self.sReplayBufferRestorePath):
            os.makedirs(self.sReplayBufferRestorePath, exist_ok=True)

    # Внутренняя функция для заполнения буфера одним шагом
    def __fill(self, uDebugLevel):
        uEpisode = tfEval(self.tuEpisode)
        bDone = self.oPlayer.next()
        uFillCount = 0

        if self.bEpisodeRating:
            if self.uBufferSize >= self.uBufferCapacity:
                self.uBufferCapacity += 256

                self.afPrevObservations = np.resize(self.afPrevObservations, (self.uBufferCapacity, self.oPlayer.oEnvironment.uObservationSize))
                self.afObservations = np.resize(self.afObservations, (self.uBufferCapacity, self.oPlayer.oEnvironment.uObservationSize))
                if self.oPlayer.oEnvironment.bDiscrete:
                    self.auActions = np.resize(self.auActions, (self.uBufferCapacity,))
                else:
                    self.afActions= np.resize(self.afActions, (self.uBufferCapacity, self.oPlayer.oEnvironment.uActionsSize))
                self.afRewards = np.resize(self.afRewards, (self.uBufferCapacity,))
                self.afRatings = np.resize(self.afRatings, (self.uBufferCapacity,))

            uIndex = self.uBufferSize

            self.afPrevObservations[uIndex] = self.oPlayer.afPrevObservation
            self.afObservations[uIndex] = self.oPlayer.afObservation
            if self.oPlayer.oEnvironment.bDiscrete:
                self.auActions[uIndex] = self.oPlayer.uAction
            else:
                self.afActions[uIndex] = self.oPlayer.afActions
            self.afRewards[uIndex] = self.oPlayer.fReward

            self.uBufferSize += 1
        else:
            if self.oPlayer.oEnvironment.bDiscrete:
                tfEval(self.tfnReplayAdd, {
                    self.tinafReplayPrevObservation: self.oPlayer.afPrevObservation,
                    self.tinafReplayObservation: self.oPlayer.afObservation,
                    self.tinauReplayAction: [self.oPlayer.uAction],
                    self.tinafReplayReward: [self.oPlayer.fReward],
                    self.tinfReplayDone: float(bDone),
                    self.tinfReplayScore: self.oPlayer.fScore,
                    self.tinuReplaySteps: self.oPlayer.uStep
                })
            else:
                tfEval(self.tfnReplayAdd, {
                    self.tinafReplayPrevObservation: self.oPlayer.afPrevObservation,
                    self.tinafReplayObservation: self.oPlayer.afObservation,
                    self.tinafReplayActions: self.oPlayer.afActions,
                    self.tinafReplayReward: [self.oPlayer.fReward],
                    self.tinfReplayDone: float(bDone),
                    self.tinfReplayScore: self.oPlayer.fScore,
                    self.tinuReplaySteps: self.oPlayer.uStep
                })
            uFillCount += 1

        if bDone:
            if self.bEpisodeRating:
                uIndex = self.uBufferSize - 1
                fLastScore = self.afRatings[uIndex] = self.afRewards[uIndex]
                while uIndex > 0:
                    uIndex -= 1
                    fLastScore = self.afRatings[uIndex] = self.afRewards[uIndex] + self.fRatingDiscountFactor * fLastScore

                for uIndex in range(self.uBufferSize - 1):
                    if self.oPlayer.oEnvironment.bDiscrete:
                        tfEval(self.tfnReplayAdd, {
                            self.tinafReplayPrevObservation: self.afPrevObservations[uIndex],
                            self.tinafReplayObservation: self.afObservations[uIndex],
                            self.tinauReplayAction: [self.auActions[uIndex]],
                            self.tinafReplayRating: [self.afRatings[uIndex]],
                            self.tinfReplayDone: 0.0,
                            self.tinfReplayScore: self.oPlayer.fScore,
                            self.tinuReplaySteps: self.oPlayer.uStep
                        })
                    else:
                        tfEval(self.tfnReplayAdd, {
                            self.tinafReplayPrevObservation: self.afPrevObservations[uIndex],
                            self.tinafReplayObservation: self.afObservations[uIndex],
                            self.tinafReplayActions: [self.afActions[uIndex]],
                            self.tinafReplayRating: [self.afRatings[uIndex]],
                            self.tinfReplayDone: 0.0,
                            self.tinfReplayScore: self.oPlayer.fScore,
                            self.tinuReplaySteps: self.oPlayer.uStep
                        })
                    uFillCount += 1

                uIndex = self.uBufferSize - 1
                if self.oPlayer.oEnvironment.bDiscrete:
                    tfEval(self.tfnReplayAdd, {
                        self.tinafReplayPrevObservation: self.afPrevObservations[uIndex],
                        self.tinafReplayObservation: self.afObservations[uIndex],
                        self.tinauReplayAction: [self.auActions[uIndex]],
                        self.tinafReplayRating: [self.afRatings[uIndex]],
                        self.tinfReplayDone: 1.0,
                        self.tinfReplayScore: self.oPlayer.fScore,
                        self.tinuReplaySteps: self.oPlayer.uStep
                    })
                else:
                    tfEval(self.tfnReplayAdd, {
                        self.tinafReplayPrevObservation: self.afPrevObservations[uIndex],
                        self.tinafReplayObservation: self.afObservations[uIndex],
                        self.tinafReplayActions: [self.afActions[uIndex]],
                        self.tinafReplayRating: [self.afRatings[uIndex]],
                        self.tinfReplayDone: 1.0,
                        self.tinfReplayScore: self.oPlayer.fScore,
                        self.tinuReplaySteps: self.oPlayer.uStep
                    })
                uFillCount += 1

                self.uBufferSize = 0

        if uDebugLevel > 0:
            uEnd, uStart, uReplayCapacity = tfEval([self.tuReplayEnd, self.tuReplayStart, self.tuReplayCapacity])

            print('\r[MEM:%s] Fill: Ep.%d:%d, Score %f, RewardAvg %f, Used %d of %d        ' % (
                getMemoryUsage(),
                uEpisode, self.oPlayer.uStep, self.oPlayer.fScore, self.oPlayer.fAverageReward,
                uEnd - uStart,
                uReplayCapacity
            ), end = '\n' if bDone else '')

        if bDone:
            self.oPlayer.reset()

        return bDone, uFillCount

    # Заполнить буфер повтора N шагами или N эпизодами, в зависимости от настройки
    def fill(self, uFillSize=1, uDebugLevel=0, bPrefill=False):
        if uDebugLevel > 1:
            print('\rFill: Processing...', end='')

        uTotalFillCount = 0

        for _ in range(uFillSize):
            bDone, uFillCount = self.__fill(uDebugLevel)
            uTotalFillCount += uFillCount

        if bPrefill:
            while not bDone:
                bDone, uFillCount = self.__fill(uDebugLevel)
                uTotalFillCount += uFillCount

        if uDebugLevel > 1:
            uEnd, uStart, uReplayCapacity = tfEval([self.tuReplayEnd, self.tuReplayStart, self.tuReplayCapacity])

            print('\nStatus: Used %d of %d from %d to %d' % (
                uEnd - uStart,
                uReplayCapacity,
                uStart % uReplayCapacity,
                uEnd % uReplayCapacity
            ))

        return uTotalFillCount, bDone

    # Выполнить инициализацию графа
    def initialize(self):
        return tfEval([self.oSummaryWriter.init(), self.tfnInitialize])

    # Сохранить модель
    def save(self):
        self.oTargetSaver.save(tfActiveSession.tfoSession, self.sTargetRestorePath)
        self.oTrainSaver.save(tfActiveSession.tfoSession, self.sTrainRestorePath)
        self.oReplayBufferSaver.save(tfActiveSession.tfoSession, self.sReplayBufferRestorePath)

    # Восстановить модель
    def restore(self):
        try:
            self.oTargetSaver.restore(tfActiveSession.tfoSession, self.sTargetRestorePath)
        except ValueError:
            pass

        try:
            self.oTrainSaver.restore(tfActiveSession.tfoSession, self.sTrainRestorePath)
        except ValueError:
            pass

        try:
            self.oReplayBufferSaver.restore(tfActiveSession.tfoSession, self.sReplayBufferRestorePath)
        except ValueError:
            return False

        return True

    # Произвести один цикл тренировки
    def train(self):
        fStartTime = time.time()
        fTotalLoss = tfEval(self.tfnTrain)
        fTimeElapsed = time.time() - fStartTime

        return fTotalLoss, fTimeElapsed

    # Опустошить все буферы записи перед закрытием
    def flush(self):
        tfEval(self.oSummaryWriter.flush())

# Тренировочный цикл

In [12]:
# Создать среду
oEnvironment = CustomEnvironment(bRender=bRender)
# Определить активный граф
with tfGraph() as tfoGraph:
    # Создать нейросеть `Актёр-цель`
    nnActor = CreateActor(oEnvironment, 1)
    # Создать виртуального игрока
    oRandomPlayer = CustomPlayer(oEnvironment, nnActor, fnSelectNoisyRandom)
    # Создать агента алгоритма `Soft Actor Critic`
    oAgent = SacAgent(oRandomPlayer,
        uReplayCapacity=uReplayCapacity,
        fRatingDiscountFactor=fRatingDiscountFactor,
        uBatchSize=uBatchSize,
        bEpisodeRating=bEpisodeRating,
        bPriorityMode=bPriorityMode,
        fTrainableTargetAverageUpdateCoef=fTrainableTargetAverageUpdateCoef,
        uLogLevel=uLogLevel,
        sLogsPath='logs/' + sName + '/',
        sRestorePath='models/' + sName + '/')

    # Начать сессию вычисления графа
    with tfSession(tfoGraph):
        # Инициализировать глобальные переменные
        tfInitGlobal()
        # Инициализировать глобальные переменные
        tfInitLocal()
        # Инициализировать агента
        oAgent.initialize()
        if not bRestore or not oAgent.restore():
            # Предварительно заполнениь буфер повтора начальными данными
            oAgent.fill(uBatchSize * 4, uDebugLevel=2, bPrefill=True)

        try:
            while True:
                # Заполнить буфер повтора шагом или эпизодом, в зависимости от настройки
                uFillCount, bDone = oAgent.fill(1, uDebugLevel=1)
                # На каждый новый шаг в буфре повтора выполнить тренировку
                while uFillCount > 0:
                    fTotalLoss = oAgent.train()
                    uFillCount -= 1
                # Сохранить модель
                if bDone:
                    oAgent.save()

        except KeyboardInterrupt:
            print('\nKeyboard Interrupt')
            pass

        finally:
            # Опустошить все буферы записи перед закрытием
            oAgent.flush()
            # Закрыть среду
            oEnvironment.close()

[MEM:507957248:3941535744] Fill: Ep.1:65, Score -101.495070, RewardAvg -0.101479, Used 65 of 131072        
[MEM:508227584:3941535744] Fill: Ep.2:111, Score -466.828336, RewardAvg -0.451310, Used 176 of 131072        
[MEM:508227584:3941535744] Fill: Ep.3:114, Score -167.294186, RewardAvg -0.164274, Used 290 of 131072        
[MEM:508227584:3941535744] Fill: Ep.4:91, Score 26.297715, RewardAvg 0.027324, Used 381 of 131072          
[MEM:508227584:3941535744] Fill: Ep.5:56, Score -91.314819, RewardAvg -0.090991, Used 437 of 131072        
[MEM:508227584:3941535744] Fill: Ep.6:88, Score -252.655468, RewardAvg -0.247945, Used 525 of 131072        
[MEM:508227584:3941535744] Fill: Ep.7:75, Score -104.780645, RewardAvg -0.103528, Used 600 of 131072        
[MEM:508264448:3941797888] Fill: Ep.8:74, Score -179.637784, RewardAvg -0.177484, Used 674 of 131072        
[MEM:508264448:3941797888] Fill: Ep.9:83, Score -115.128642, RewardAvg -0.114426, Used 757 of 131072        
[MEM:508403712:39415